In [4]:
import graspy
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
#uncomment when placed into model folder not doc
#from .base import BaseGraphEstimator, _calculate_p

from scipy.stats import bernoulli 
import pandas as pd 

from graspy.simulations import sbm, er_np, er_nm
from graspy.plot import heatmap

from graspy.models.base import BaseGraphEstimator 
from graspy.utils.utils import (
    augment_diagonal,
    cartprod,
    import_graph,
    is_unweighted,
    remove_loops,
    symmetrize,
)

from PyNonpar.twosample import wilcoxon_mann_whitney_test

from seaborn import *
from dfply import *

In [8]:
class SIEMEstimator(BaseGraphEstimator):
    r"""
    Stochastic Block Model 
    The stochastic block model (SBM) represents each node as belonging to a block 
    (or community). For a given potential edge between node :math:`i` and :math:`j`, 
    the probability of an edge existing is specified by the block that nodes :math:`i`
    and :math:`j` belong to:
    :math:`P_{ij} = B_{\tau_i \tau_j}`
    where :math:`B \in \mathbb{[0, 1]}^{K x K}` and :math:`\tau` is an `n\_nodes` 
    length vector specifying which block each node belongs to. 
    Read more in the :ref:`tutorials <models_tutorials>`
    Parameters
    ----------
    directed : boolean, optional (default=True)
        Whether to treat the input graph as directed. Even if a directed graph is inupt, 
        this determines whether to force symmetry upon the block probability matrix fit
        for the SBM. It will also determine whether graphs sampled from the model are 
        directed. 
    loops : boolean, optional (default=False)
        Whether to allow entries on the diagonal of the adjacency matrix, i.e. loops in 
        the graph where a node connects to itself. 
    n_components : int, optional (default=None)
        Desired dimensionality of embedding for clustering to find communities.
        ``n_components`` must be ``< min(X.shape)``. If None, then optimal dimensions 
        will be chosen by :func:`~graspy.embed.select_dimension``.
    min_comm : int, optional (default=1)
        The minimum number of communities (blocks) to consider. 
    max_comm : int, optional (default=10)
        The maximum number of communities (blocks) to consider (inclusive).
    cluster_kws : dict, optional (default={})
        Additional kwargs passed down to :class:`~graspy.cluster.GaussianCluster`
    embed_kws : dict, optional (default={})
        Additional kwargs passed down to :class:`~graspy.embed.AdjacencySpectralEmbed`
    Attributes
    ----------
    block_p_ : np.ndarray, shape (n_blocks, n_blocks)
        The block probability matrix :math:`B`, where the element :math:`B_{i, j}`
        represents the probability of an edge between block :math:`i` and block 
        :math:`j`.
    p_mat_ : np.ndarray, shape (n_verts, n_verts)
        Probability matrix :math:`P` for the fit model, from which graphs could be
        sampled.
    vertex_assignments_ : np.ndarray, shape (n_verts)
        A vector of integer labels corresponding to the predicted block that each node 
        belongs to if ``y`` was not passed during the call to ``fit``. 
    block_weights_ : np.ndarray, shape (n_blocks)
        Contains the proportion of nodes that belong to each block in the fit model.
    See also
    --------
    graspy.simulations.siem
    References
    ----------
    .. [1]  Holland, P. W., Laskey, K. B., & Leinhardt, S. (1983). Stochastic
            blockmodels: First steps. Social networks, 5(2), 109-137.
    """
    def __init__(
        self,
        directed=True,
        loops=False,
#         n_components=None,
#         min_comm=1,
#         max_comm=10,
#         cluster_kws={},
#         embed_kws={},
    ):
        super().__init__(directed=directed, loops=loops)
        self.model = {}
#         _check_common_inputs(n_components, min_comm, max_comm, cluster_kws, embed_kws)
#         self.cluster_kws = cluster_kws
#         self.n_components = n_components
#         self.min_comm = min_comm
#         self.max_comm = max_comm
#         self.embed_kws = embed_kws
#     def _estimate_assignments(self, graph):
#         """
#         Do some kind of clustering algorithm to estimate communities
#         There are many ways to do this, here is one
#         """
#         embed_graph = augment_diagonal(graph)
#         latent = AdjacencySpectralEmbed(
#             n_components=self.n_components, **self.embed_kws
#         ).fit_transform(embed_graph)
#         if isinstance(latent, tuple):
#             latent = np.concatenate(latent, axis=1)
#         gc = GaussianCluster(
#             min_components=self.min_comm,
#             max_components=self.max_comm,
#             **self.cluster_kws
#         )
#         vertex_assignments = gc.fit_predict(latent)
#         self.vertex_assignments_ = vertex_assignments
    def fit(self, graph, edge_comm, weighted):
        """
        Fit the SIEM to a graph
        Parameters
        ----------
        graph : array_like or networkx.Graph [nxn]
            Input graph to fit
        edge_comm : 2d list of k tuples (k_communities)
            Categorical labels for the block assignments of the graph
        weighted: boolean or float
            Boolean: True - do nothing or False - ensure everything is 0 or 1
            Float: binarize and use float as cutoff
        """
        #checks
        n = graph.shape[0]
        if not(isinstance(graph, (nx.Graph, nx.DiGraph, nx.MultiGraph, nx.MultiDiGraph, np.ndarray))):
            msg = "graph must be a np.array or networkx.Graph"
            raise TypeError(msg)
        if not isinstance(edge_comm, list):
            msg = "Edge_comm must be a list"
            raise TypeError(msg)
        if len(edge_comm) >= n:
            msg = "warning more communities than n vertices"
            print(msg)
        if len(edge_comm) > n**2:
            msg = "Too many communities for this graph"
            raise TypeError(msg)
        if not(type(weighted) == bool or type(weighted) == float): 
            msg = "weighted must be a boolean or float"
            raise TypeError(msg)
        graph = import_graph(graph)
        if isinstance(weighted,float): 
            graph = 1*(graph>weighted)
        
        if weighted == False: 
            if not np.array_equal(graph, graph.astype(bool)):
                msg = "graph of weighted = False must have binary inputs"
                raise TypeError(msg)
                
                
        for i in range(0,len(edge_comm)):
            comms = []
            for x in edge_comm[i]:
                comms.append(graph[x])
                
            self.model[i] = np.array(comms)
        return
    
    def test(self, method="wilcox", alternative="greater", alpha=.05):
        """
        Test whether there exists a difference in the edge weights from one
        community to another. This method should be called after fitting.
        Parameters
        ----------
        method: str, optional (default="hodge")
            The approach to use for testing. Options are:
                "hodge": The Hodge-Lehmann Estimator. Number of unique communities 
                in `self.edge_comm` should be 2. The parameter `alternative` will specify the 
                alternative hypothesis, where the edge communities will be considered in the order
                given by `self.edge_comm.keys()`.
        alternative: str, optional (default="greater")
            The alternative hypothesis for the test. This option is supported only
            in the event that the number of edge communities is 2. A warning will be passed
            if the number of communities exceeds 2, and an alternative hypothesis is specified.
            The possible options for an alternative hypothesis are "greater" (one-sided),
            "less" (one-sided), and "neq" (two-tailed).
            In the case that more than two communities are present, 
            `alternative` should be specified as `None`.
        alpha: float (default=.05)
            The significance 
        """
        if not self.model:
            raise ValueError("You have not fit a model yet.")
        if (method == "wilcox" and len(self.edge_comm) != 2):
            msg = """wilcox rank-sum test can only be applied when the number
            of unique edge-communities is 2. Your data has {}.""".format(len(self.edge_comm))
            raise ValueError(msg)
        if not (method in ["wilcox"]):
            raise ValueError("You have not passed a supported method.")
        if (method == "wilcox" and alternative not in ["greater", "less", "neq"]):
            msg = "You have not specified a valid alternative. Taking to be neq (two-tailed)..."
            warnings.warn(msg)
            alternative = "neq"
        if method not in ["wilcox"] and alternative not None:
            msg = """You have provided an alternative hypothesis, but the method
            you have selected does not support an alternative hypothesis. Taking to be None..."""
            warnings.warn(msg)
            alternative = None
        if method is "wilcox":
            keys = self.model.keys()
            test = wilcoxon_mann_whitney_test(self.model[keys[0]], self.model[keys[1]],
                                              alternative=alternative, alpha=alpha)
        return test
        
    
def siem(n, p,edge_comm, directed=False, loops=False, wt=None, wtargs=None):
    """
    Samples a graph from the structured independent edge model (SIEM) 
    SIEM produces a graph with specified communities, in which each community can
    have different sizes and edge probabilities. 
    Read more in the :ref:`tutorials <simulations_tutorials>`
    Parameters
    ----------
    n: int
        Number of vertices
    p: list of int of length K (k_communities)
        Probability of an edge existing within the corresponding communities, where p[i] indicates 
        the probability of an edge existing in the edge_comm[i]
    edge_comm: 2d list of K tuples (k_communities)
        tuple is the indices for the edge within the kth community.
    directed: boolean, optional (default=False)
        If False, output adjacency matrix will be symmetric. Otherwise, output adjacency
        matrix will be asymmetric.
    loops: boolean, optional (default=False)
        If False, no edges will be sampled in the diagonal. Otherwise, edges
        are sampled in the diagonal.
    wt: object or list of K objects (k_communities)
        if Wt is an object, a weight function to use globally over
        the siem for assigning weights. If Wt is a list, a weight function for each of
        the edge communities to use for connection strengths Wt[i] corresponds to the weight function
        for edge community i. Default of None results in a binary graph
    wtargs: dictionary or array-like, shape (k_communities)
        if Wt is an object, Wtargs corresponds to the trailing arguments
        to pass to the weight function. If Wt is an array-like, Wtargs[i, j] 
        corresponds to trailing arguments to pass to Wt[i, j].
    return_labels: boolean, optional (default = False)
        IF True, returns the edge-communities as well
    References
    ----------
    Returns
    -------
    A: ndarray, shape (sum(n), sum(n))
        Sampled adjacency matrix
    T: returns the edge-communities if return_labels == True
    Examples
    --------
    >>> np.random.seed(1)
    >>> n = [3, 3]
    >>> p = [[0.5, 0.1], [0.1, 0.5]]
    To sample a binary 2-block SBM graph:
    >>> sbm(n, p)
    array([[0., 0., 1., 0., 0., 0.],
           [0., 0., 1., 0., 0., 1.],
           [1., 1., 0., 0., 0., 0.],
           [0., 0., 0., 0., 1., 0.],
           [0., 0., 0., 1., 0., 0.],
           [0., 1., 0., 0., 0., 0.]])
    To sample a weighted 2-block SBM graph with Poisson(2) distribution:
    >>> wt = np.random.poisson
    >>> wtargs = dict(lam=2)
    >>> sbm(n, p, wt=wt, wtargs=wtargs)
    array([[0., 4., 0., 1., 0., 0.],
           [4., 0., 0., 0., 0., 2.],
           [0., 0., 0., 0., 0., 0.],
           [1., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0.],
           [0., 2., 0., 0., 0., 0.]])
    """
    # Check n
    if not isinstance(n, (int)):
        msg = "n must be a int, not {}.".format(type(n))
        raise TypeError(msg)
    # Check edge_comm 
    if not isinstance(edge_comm, (list)):
        msg = "edge_comm must be a 2d list of length k."
        raise TypeError(msg)
    else: 
        for i in range(len(edge_comm)):
            for x in edge_comm[i]:
                if not (len(x)==2 and isinstance (x,tuple)):
                    msg = "The edge_comm list must contain tuples of 2 elements."
                    raise TypeError(msg)
        #edge_comm = np.array(edge_comm)
        #generate temporary adjacency matrix to check upper triangular?
        if (directed == True) and (loops == True):
            if not(sum(len(x) for x in edge_comm) <= n**2) : 
                msg = "Edge Communities and Number of Vertices Do Not Agree, {} must be <= n**2 !".format(sum(len(x) for x in edge_comm))
                raise ValueError(msg)
        elif (directed == True) and (loops == False): 
            if not(sum(len(x) for x in edge_comm) <= n*(n-1)/2): 
                msg = "Edge Communities and Number of Vertices Do Not Agree, {} must be <= n(n-1)/2!".format(sum(len(x) for x in edge_comm))
                raise ValueError(msg)            
        elif (directed == False) and (loops == True):
            #check symmetry ?
            if not(sum(len(x) for x in edge_comm) <= n*(n-1)) : 
                msg = "Edge Communities and Number of Vertices Do Not Agree, {} must be <= n(n-1)!".format(sum(len(x) for x in edge_comm))
                raise ValueError(msg)            
        elif (directed == False) and (loops == False): 
            #check symmetry ?
            if not(sum(len(x) for x in edge_comm) <= n**2/2) : 
                msg = "Edge Communities and Number of Vertices Do Not Agree, {} must be <= n**2/2!".format(sum(len(x) for x in edge_comm))
                raise ValueError(msg)            
    # Check p
    if not isinstance(p, (list, np.ndarray)):
        msg = "p must be a list or np.array, not {}.".format(type(p))
        raise TypeError(msg)
    else:
        p = np.array(p)
        if not np.issubdtype(p.dtype, np.number):
            msg = "There are non-numeric elements in p"
            raise ValueError(msg)
        elif np.any(p < 0) or np.any(p > 1):
            msg = "Values in p must be in between 0 and 1."
            raise ValueError(msg)
        elif len(p) != len(edge_comm):
            msg = "# of Probabilities and # of Communities Don't Match Up"
            raise ValueError(msg)
    # Check wt and wtargs
    if (wt is not None) and (wtargs is not None): 
        if callable(wt):
            #extend the function to size of k 
            wt = np.full(len(edge_comm), wt, dtype=object)
            wtargs = np.full(len(edge_comm), wtargs, dtype=object)
        elif type(wt) == list:
            if all(callable(x) for x in wt): 
                # if not object, check dimensions
                if len(wt) != (len(edge_comm)):
                    msg = "wt must have size k, not {}".format(len(wt))
                    raise ValueError(msg)
                if len(wtargs) != (len(edge_comm)):
                    msg = "wtargs must have size k , not {}".format(len(wtargs))
                    raise ValueError(msg)
                # check if each element is a function
                for element in wt.ravel():
                    if not callable(element):
                        msg = "{} is not a callable function.".format(element)
                        raise TypeError(msg)   
            else: 
                msg = "list must contain all callable objects"
                raise ValueError(msg)
        else:
            msg = "wt must be a callable object or list of callable objects"
            raise ValueError(msg)

    K = len(edge_comm) # the number of communities
    # End Checks, begin simulation
    A = np.zeros((n,n))

    # list of lists of 2-d tuples version
    for i in range(0, K):
        #sample bernoulli at once
        rvs = bernoulli.rvs(p[i],size = len(edge_comm[i]))
        #iterate over each index
        for x in range(len(edge_comm[i])):
            #set each equal to bernoulli 
            A[edge_comm[i][x]] = rvs[x]
            #adjust adjacency matrix with any weight args. 
            if (wt is not None) and (wtargs is not None): 
                    A[edge_comm[i][x]] = A[edge_comm[i][x]]*wt[i](**wtargs[i])
    
    if not directed:
        A = A + A.T - np.diag(A)
        
    return A


IndentationError: expected an indented block (<ipython-input-8-17ddec809be6>, line 164)